Udacity Student Data Analysis
=========================
## Preparing data for Analysis  
### Load Data from CSVs

In [1]:
import unicodecsv

In [2]:
def csv_to_list(filename: str) -> list:
    # takes a string filename to a csv and returns its data as a list
    with open(filename,'rb') as f_in:
        reader = unicodecsv.DictReader(f_in)
        return list(reader)

In [3]:
engagement_filename = 'data/daily_engagement.csv'
enrollments_filename = 'data/enrollments.csv'
submissions_filename = 'data/project_submissions.csv'

engagements = csv_to_list(engagement_filename)
enrollments = csv_to_list(enrollments_filename)
submissions = csv_to_list(submissions_filename)

In [4]:
def print_lists(ele=0):
    print("Engagements :",engagements[ele])
    print()
    print("Enrollments :",enrollments[ele])
    print()
    print("Submissions :",submissions[ele])
print_lists()

Engagements : OrderedDict([('acct', '0'), ('utc_date', '2015-01-09'), ('num_courses_visited', '1.0'), ('total_minutes_visited', '11.6793745'), ('lessons_completed', '0.0'), ('projects_completed', '0.0')])

Enrollments : OrderedDict([('account_key', '448'), ('status', 'canceled'), ('join_date', '2014-11-10'), ('cancel_date', '2015-01-14'), ('days_to_cancel', '65'), ('is_udacity', 'True'), ('is_canceled', 'True')])

Submissions : OrderedDict([('creation_date', '2015-01-14'), ('completion_date', '2015-01-16'), ('assigned_rating', 'UNGRADED'), ('account_key', '256'), ('lesson_key', '3176718735'), ('processing_state', 'EVALUATED')])


### Convert Ordered dict to dict

In [5]:
def convert_to_dict():
    for ele in range(len(engagements)):
        engagements[ele] = dict(engagements[ele])
    for ele in range(len(enrollments)):
        enrollments[ele] = dict(enrollments[ele])
    for ele in range(len(submissions)):
        submissions[ele] = dict(submissions[ele])
convert_to_dict()

In [6]:
print_lists()

Engagements : {'acct': '0', 'utc_date': '2015-01-09', 'num_courses_visited': '1.0', 'total_minutes_visited': '11.6793745', 'lessons_completed': '0.0', 'projects_completed': '0.0'}

Enrollments : {'account_key': '448', 'status': 'canceled', 'join_date': '2014-11-10', 'cancel_date': '2015-01-14', 'days_to_cancel': '65', 'is_udacity': 'True', 'is_canceled': 'True'}

Submissions : {'creation_date': '2015-01-14', 'completion_date': '2015-01-16', 'assigned_rating': 'UNGRADED', 'account_key': '256', 'lesson_key': '3176718735', 'processing_state': 'EVALUATED'}


### Fix Datatypes

In [7]:
from datetime import datetime as dt

In [8]:
if(type(enrollments[0]['status']) == str):
    for enrollment in enrollments:
        enrollment['status'] = enrollment['status']=='current'
        enrollment['join_date'] = None if enrollment['join_date']=='' else dt.strptime(enrollment['join_date'], '%Y-%m-%d')
        enrollment['cancel_date'] = None if enrollment['cancel_date']=='' else dt.strptime(enrollment['cancel_date'], '%Y-%m-%d')
        enrollment['days_to_cancel'] = None if enrollment['days_to_cancel']=='' else int(enrollment['days_to_cancel'])
        enrollment['is_udacity'] = enrollment['is_udacity']=='True'
        enrollment['is_canceled'] = enrollment['is_canceled']=='True'
    print("Parsed")
else: print("Already Parsed")

Parsed


In [9]:
if(type(engagements[0]['utc_date']) == str):
    for engagement in engagements:
        engagement['utc_date'] = None if engagement['utc_date']=='' else dt.strptime(engagement['utc_date'], '%Y-%m-%d')
        engagement['num_courses_visited'] = int(float(engagement['num_courses_visited']))
        engagement['total_minutes_visited'] = float(engagement['total_minutes_visited'])
        engagement['lessons_completed'] = int(float(engagement['lessons_completed']))
        engagement['projects_completed'] = int(float(engagement['projects_completed']))
    print("Parsed")
else: print("Already Parsed")

Parsed


In [10]:
if(type(submissions[0]['creation_date']) == str):
    for submission in submissions:
        submission['creation_date'] = None if submission['creation_date']=='' else dt.strptime(submission['creation_date'], '%Y-%m-%d')
        submission['completion_date'] = None if submission['completion_date']=='' else dt.strptime(submission['completion_date'], '%Y-%m-%d') 
    print("Parsed")
else: print("Already Parsed")

Parsed


In [11]:
print_lists(0)

Engagements : {'acct': '0', 'utc_date': datetime.datetime(2015, 1, 9, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 11.6793745, 'lessons_completed': 0, 'projects_completed': 0}

Enrollments : {'account_key': '448', 'status': False, 'join_date': datetime.datetime(2014, 11, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0), 'days_to_cancel': 65, 'is_udacity': True, 'is_canceled': True}

Submissions : {'creation_date': datetime.datetime(2015, 1, 14, 0, 0), 'completion_date': datetime.datetime(2015, 1, 16, 0, 0), 'assigned_rating': 'UNGRADED', 'account_key': '256', 'lesson_key': '3176718735', 'processing_state': 'EVALUATED'}


## Investigating the  data
* Find the total number of rows and the number of unique students (account keys) in each table.

In [12]:
# No of rows
print("Engagements :",len(engagements))
print("Enrollments :",len(enrollments))
print("Submissions :",len(submissions))

Engagements : 136240
Enrollments : 1640
Submissions : 3642


### Problems in the data
#### Rename the "acct" column in the `daily_engagement` table to "account_key".

In [14]:
# Rename the "acct" column in the daily_engagement table to "account_key".
if(engagements[0].get('acct')):
    for engagement in engagements:
        engagement['account_key'] = engagement['acct']
        engagement.pop('acct')

#### get_unique_students `Function`
Takes in data (a list of dicts) and returns a set of unique student's account numbers

In [15]:
def get_unique_students(data):
    unique_students_set = set()
    for ele in data:
        unique_students_set.add(ele['account_key'])
    return unique_students_set

#### get_student_records `Function`
Takes in data(a list of dicts), account_key(string) and returns a list of records of that student

In [16]:
def get_student_records(data, key, popitms = None, key_type='account_key'):
    # assume popitms is a list of strings
    student_records = list()
    for ele in data:
        if ele[key_type]==key:
            if not popitms:
                student_records.append(ele)
            else:
                poped_ele = dict()
                poped_ele.update(ele)
                poped_ele.pop(popitms)
                student_records.append(poped_ele)
    return student_records

#### Find students who are reported in enrollment but not in engagement
#### run_unique_test `Function`
Takes in two lists. First List is enrollments. Second is Engagements.
Each list is a list of dicts.

In [17]:
def run_unique_test(enrollments,engagements):
    enrollment_students = get_unique_students(enrollments)
    engagement_students = get_unique_students(engagements)

    non_engaging_students = enrollment_students-engagement_students

    print("students who are reported in enrollment but not in engagement")
    print(non_engaging_students, "\n")
    print("No of Enrollment records: {}, No of Engagement records: {}".format(len(enrollments), len(engagements) ) )
    print("No of Unique Students in Enrollment records: {}\nNo of Unique Students in Engagement records: {}".format(len(enrollment_students), len(engagement_students) ) )
    print("No of non engaging students:", len(non_engaging_students) )
    print("\nCheck to see if there are any engaging students that are not reported in Enrollment records:")
    if not len(engagement_students-enrollment_students):
        print("None")
    else:
        print("There are any engaging students that are not reported in Enrollment records" )
    return non_engaging_students
non_engaging_students=run_unique_test(enrollments,engagements)

students who are reported in enrollment but not in engagement
{'889', '728', '914', '1125', '902', '819', '997', '799', '926', '1222', '733', '707', '725', '1148', '1129', '817', '1237', '964', '1069', '1044', '1186', '750', '968', '766', '1273', '1120', '1145', '1238', '996', '789', '1025', '1155', '737', '1304', '871', '802', '870', '1241', '841', '1219', '981', '1284', '1291', '739', '1218', '803', '1010', '717', '749', '1190', '1086', '1079', '1270', '884', '1063', '878', '1213', '1191', '711', '727', '664', '654', '1171', '1101', '875'} 

No of Enrollment records: 1640, No of Engagement records: 136240
No of Unique Students in Enrollment records: 1302
No of Unique Students in Engagement records: 1237
No of non engaging students: 65

Check to see if there are any engaging students that are not reported in Enrollment records:
None


#### Observe the data of these students in enrollment data

**`Observations :`**
* Udacity test accounts were found to be part of problem. Need to remove those accounts from analysis.

In [18]:
# Observe the data of these students in enrollment data
# 1010

# Step 1: output datapoints of a user by the account_key

for student in non_engaging_students:
#student = '1219'
    for enrollment in enrollments:
        if enrollment['account_key'] == student:
            print(enrollment)


{'account_key': '889', 'status': False, 'join_date': datetime.datetime(2015, 8, 11, 0, 0), 'cancel_date': datetime.datetime(2015, 8, 11, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}
{'account_key': '728', 'status': False, 'join_date': datetime.datetime(2014, 12, 5, 0, 0), 'cancel_date': datetime.datetime(2014, 12, 5, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}
{'account_key': '914', 'status': False, 'join_date': datetime.datetime(2015, 4, 2, 0, 0), 'cancel_date': datetime.datetime(2015, 4, 2, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}
{'account_key': '914', 'status': False, 'join_date': datetime.datetime(2015, 7, 17, 0, 0), 'cancel_date': datetime.datetime(2015, 7, 17, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}
{'account_key': '1125', 'status': False, 'join_date': datetime.datetime(2015, 5, 31, 0, 0), 'cancel_date': datetime.datetime(2015, 5, 31, 0, 0), 'days_to_cancel': 0, 'is_udac

In [19]:
# Step 2: Find a solution to remove Udacity test accounts from further analysis
udacity_test_accounts = set()
non_udacity_enrollments = list()
non_udacity_engagements = list()
non_udacity_submissions = list()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
    else:
        temp = dict()
        temp.update(enrollment)
        temp.pop('is_udacity')
        non_udacity_enrollments.append(temp)
        
for engagement in engagements:
    if engagement['account_key'] not in udacity_test_accounts:
        non_udacity_engagements.append(engagement)
        
for submission in submissions:
    if submission['account_key'] not in udacity_test_accounts:
        non_udacity_submissions.append(submission)
        
print(len(udacity_test_accounts))
print(len(non_udacity_enrollments)) # not number of unique students. This is just number of records
print(len(non_udacity_engagements))
print(len(non_udacity_submissions))
non_engaging_students = run_unique_test(non_udacity_enrollments,non_udacity_engagements)

6
1622
135656
3634
students who are reported in enrollment but not in engagement
{'889', '728', '914', '1125', '902', '819', '997', '799', '926', '1222', '733', '707', '725', '1148', '1129', '817', '1237', '964', '1186', '1044', '750', '968', '766', '1273', '1120', '1145', '1238', '996', '789', '1025', '1155', '737', '871', '802', '870', '1241', '841', '1219', '981', '1284', '1291', '739', '1218', '803', '1010', '717', '749', '1190', '1086', '1079', '1270', '884', '1063', '878', '1213', '1191', '711', '727', '664', '654', '1171', '875'} 

No of Enrollment records: 1622, No of Engagement records: 135656
No of Unique Students in Enrollment records: 1296
No of Unique Students in Engagement records: 1234
No of non engaging students: 62

Check to see if there are any engaging students that are not reported in Enrollment records:
None


#### Datasets to work on from here
These data excludes Udacity test account records.  
`Var Names`
* non_udacity_enrollments
* non_udacity_engagements
* non_udacity_submissions  

All `keys` are same as the data sets they are derived from except:
* `'is_udacity'` is not in non_udacity_enrollments
* `'acct'` is renamed to 'account_key'

## Analysis 1:
Create a dictionary named `paid_students` containing all students who either
haven't canceled yet or who remained enrolled for more than 7 days. The keys
should be account keys, and the values should be the date the student enrolled.

In [20]:
# paid_students is a dict: key- 'account_key' , value- list of enrollment dates
paid_students = dict()
unique_keys = get_unique_students(non_udacity_enrollments)

# Method 1: slower method but more flexible
"""for key in unique_keys:
    records = get_student_records(non_udacity_enrollments, key)
    for record in records:
        #print(record)
        if not record['is_canceled'] or record['days_to_cancel']>7:
            paid_students[record['account_key']]=record['join_date']"""

# Method 2: faster method
for enrollment in non_udacity_enrollments:
    if not enrollment['is_canceled'] or enrollment['days_to_cancel']>7:
        if enrollment['account_key'] not in paid_students:
            paid_students[enrollment['account_key']] = list()
        paid_students[enrollment['account_key']].append(enrollment['join_date'])
        
print(len(paid_students))

#for std in paid_students:
print('584', max(paid_students['584']))
print("-------------------------------------------")
for record in get_student_records(non_udacity_enrollments, '584'):
    print(record)
print()

995
584 2015-03-10 00:00:00
-------------------------------------------
{'account_key': '584', 'status': False, 'join_date': datetime.datetime(2014, 11, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0), 'days_to_cancel': 65, 'is_canceled': True}
{'account_key': '584', 'status': False, 'join_date': datetime.datetime(2015, 1, 14, 0, 0), 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 55, 'is_canceled': True}
{'account_key': '584', 'status': True, 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_canceled': False}



Create a list of engagement records containing only data for paid students during their first week

#### ndays_engagement `Function`

In [51]:
from datetime import timedelta
def ndays_engagement(engagement_data, n=7) -> list:
    ndays_engagement_records = list()
    for data in engagement_data:
        working_key = data['account_key']
        if working_key in paid_students:
            from_date = max(paid_students[working_key])
            if data['utc_date'] < from_date + timedelta(days=n) and data['utc_date'] >= from_date:
                ndays_engagement_records.append(data)
    return ndays_engagement_records
# first_week_paid_engagements is a list or dicts(records)
first_week_paid_engagements = ndays_engagement(non_udacity_engagements)
len(first_week_paid_engagements)

6919

#### reorganize `non_udacity_engagements`  
reorganize the data in non_udacity_engagements **from**  
a list of dicts - each dict is a record of engagement **to**  
a dict of dicts - each key of the main dict contains all the records corresponding to that key

In [52]:
# create a temp variable to delete later
#reorg = first_week_engagements
#first_week_engagements = None
reorg = dict()
for key in paid_students:
    #print(key)
    reorg[key] = (get_student_records(first_week_paid_engagements, key , popitms = ('account_key') ) )
#reorg['429']
first_week_paid_engagements = reorg
del reorg

#### reorg new code

In [52]:
# create a temp variable to delete later
#reorg = first_week_engagements
#first_week_engagements = None
def regroup_records_by_key(data, group_by_key):
    # step 1 : do a key check. The key must be a valid key of the given data but cant ...
    reorg = dict()
    for key in paid_students:
        #print(key)
        reorg[key] = (get_student_records(first_week_paid_engagements, key , popitms = (group_by_key) ) )
#reorg['429']
first_week_paid_engagements = reorg
del reorg

## Data Exploration

In [55]:
total_minutes_by_account_first_week = dict()

for account_key, engagement_for_student in first_week_paid_engagements.items():
    total_mins = 0
    for engagement in engagement_for_student:
        total_mins += engagement['total_minutes_visited']
    total_minutes_by_account_first_week[account_key] = total_mins
    
total_mins = list(total_minutes_by_account_first_week.values())
print(total_minutes_by_account_first_week)

{'429': 0.0, '60': 0.0, '369': 42.3843405, '322': 0.0, '336': 278.6769970004, '553': 507.1024986672999, '584': 0.0, '458': 0.0, '1058': 0.0, '206': 633.1473566663, '45': 0.0, '315': 657.0126268326, '604': 1890.9937024994, '204': 686.7681608344001, '408': 59.7725306667, '51': 0.0, '323': 397.24734133329997, '568': 28.9028346666, '130': 0.0, '550': 18.9512853333, '44': 0.0, '198': 809.546996001, '368': 817.9997333323399, '440': 45.9874008334, '57': 91.3514995, '1090': 0.0, '541': 20.879544833329998, '756': 0.0, '101': 148.7306783333, '800': 122.87407133333, '702': 0.0, '38': 0.0, '215': 662.64059800003, '744': 374.5761038331, '217': 0.0, '1211': 0.0, '53': 401.97479050029995, '283': 269.194508, '338': 13.2610473333, '579': 298.253430333, '627': 1047.4820013337, '253': 0.0, '178': 783.3383063341, '822': 2.30081433333, '96': 45.3963998333, '123': 0.0, '620': 557.5625511664, '1204': 291.7782771666, '195': 1166.4244791677, '354': 609.2861186673, '114': 186.94018933324, '394': 808.5665558337,

In [56]:
import numpy as np

print('Mean', np.mean(total_mins))
print('Standard Deviation', np.std(total_mins))
print('Max', np.max(total_mins))
print('Min', np.min(total_mins))
print('Minutes in a week :',7*24*60)

Mean 306.70832675342825
Standard Deviation 412.99693340852957
Max 3564.7332644989997
Min 0.0
Minutes in a week : 10080


In [57]:
total_lessons_by_account_first_week = dict()

for account_key, engagement_for_student in first_week_paid_engagements.items():
    total_lessons = 0
    for engagement in engagement_for_student:
        total_lessons += engagement['lessons_completed']
    total_lessons_by_account_first_week[account_key] = total_lessons
    
total_lessons = list(total_lessons_by_account_first_week.values())
print(total_lessons_by_account_first_week)

{'429': 0, '60': 0, '369': 0, '322': 0, '336': 0, '553': 2, '584': 0, '458': 0, '1058': 0, '206': 3, '45': 0, '315': 3, '604': 8, '204': 2, '408': 0, '51': 0, '323': 2, '568': 0, '130': 0, '550': 0, '44': 0, '198': 1, '368': 4, '440': 0, '57': 1, '1090': 0, '541': 0, '756': 0, '101': 0, '800': 1, '702': 0, '38': 0, '215': 1, '744': 1, '217': 0, '1211': 0, '53': 3, '283': 0, '338': 1, '579': 1, '627': 6, '253': 0, '178': 6, '822': 0, '96': 0, '123': 0, '620': 1, '1204': 2, '195': 4, '354': 3, '114': 1, '394': 2, '1253': 0, '504': 3, '537': 0, '72': 7, '268': 5, '339': 0, '1244': 0, '752': 0, '194': 1, '520': 0, '102': 0, '367': 1, '258': 3, '522': 0, '329': 4, '225': 1, '612': 0, '150': 1, '179': 5, '595': 2, '587': 0, '439': 2, '428': 2, '18': 0, '544': 6, '10': 0, '913': 0, '23': 6, '576': 2, '431': 0, '223': 0, '480': 0, '525': 0, '1111': 0, '1107': 0, '577': 0, '92': 3, '335': 2, '275': 0, '245': 2, '135': 2, '37': 5, '186': 2, '389': 0, '693': 0, '88': 0, '983': 0, '1118': 1, '499'

In [58]:
print('Mean', np.mean(total_lessons))
print('Standard Deviation', np.std(total_lessons))
print('Max', np.max(total_lessons))
print('Min', np.min(total_lessons))

Mean 1.636180904522613
Standard Deviation 3.0025612998294227
Max 36
Min 0
